In [5]:
!python3 -m pip install requests beautifulsoup4
!python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
import requests
from bs4 import BeautifulSoup

# Fetch the HTML page
url = "https://birdsoftheworld.org/bow/species"  # Replace with your actual URL
response = requests.get(url)

# Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <li> elements with class "TaxonomyTree-card"
elements = soup.find_all('li', class_='TaxonomyTree-card')

# Print or process the elements
for element in elements:
    print(element.text.strip())  # Print the text inside each <li>


/Users/michael/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
len(elements)

0

In [ ]:
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup

# Load DataFrame (Replace with your actual DataFrame loading method)
df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\Cleaned_Clements-v2024-October-2024-rev.csv")  # Ensure your DataFrame has 'category', 'species_code', and 'scientific name' columns

# Create a folder to store images
os.makedirs(f"BOW_Images/Drawings", exist_ok=True)

session = requests.Session()  # Keep the connection open

# Example request using the session
response = session.get("https://example.com")

# Iterate over the DataFrame
for index, row in df.iterrows():
    if str(row["category"]).lower() == "family":
        family = row["scientific name"]
        sortnumber = str(row["sort v2024"]).split(".")[0].zfill(5)
        # Create a folder to store images
        os.makedirs(f"BOW_Images/Drawings/{sortnumber}_{family}", exist_ok=True)

    if row["sort v2024"] < 3401:
        continue

    if str(row["category"]).lower() not in ["species", "subspecies","group (monotypic)"]:
        continue  # Skip non-species and non-subspecies entries

    species_code = row["species_code"]
    url = f"https://birdsoftheworld.org/bow/species/{species_code}/cur/multimedia?media=illustrations"  # Replace with the actual website pattern
    response = session.get(url)

    # Check if request was successful
    if response.status_code != 200:
        continue

    # Parse the HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Check for <h1 class="Heading--h1 Heading Heading--thin">Multimedia</h1>
    multimedia_header = soup.find("h1", class_="Heading--h1 Heading Heading--thin")
    if not multimedia_header or multimedia_header.text.strip() != "Multimedia":
        continue  # Skip if "Multimedia" section is not found

    # Find all image containers
    image_elements = soup.find_all("a", class_="CarouselResponsive-slide")

    for img_elem in image_elements:
        image_id = img_elem.get("data-asset-id")
        image_url = img_elem.get("data-asset-src")
        caption = img_elem.get("data-asset-caption", "No caption").split(" ")[:3]
        caption = " ".join(caption)
        comname = img_elem.get("data-asset-comname", "Unknown")
        sciname = img_elem.get("data-asset-sciname", "Unknown")

        if not image_url:
            continue  # Skip if no image URL

        # Append /2400 to get high-resolution image
        full_image_url = f"{image_url}/2400"

        # Try to find the scientific name in the original DataFrame
        matching_row = df[df["scientific name"] == sciname]
        if not matching_row.empty:
            filename_species_code = matching_row.iloc[0]["species_code"]
        else:
            filename_species_code = species_code  # Fallback to the original species_code


        # Save the image
        image_filename = f"{filename_species_code}_{sciname}_{comname}_{caption}_{image_id}"
        invalid_chars = '<>:;"/\\|?.*'
        for char in invalid_chars:
            image_filename = image_filename.replace(char, "")  # Remove each invalid character
        image_filename = image_filename.strip()
        image_filename = f"BOW_Images/Drawings/{sortnumber}_{family}/{image_filename}.jpg"

        
        img_response = session.get(full_image_url)

        if img_response.status_code == 200:
            with open(image_filename, "wb") as img_file:
                img_file.write(img_response.content)
            print(f"        Downloaded: {comname}")
        else:
            print(f"Failed to download: {full_image_url}")

        """
        # Optionally, store metadata (e.g., in a CSV or JSON)
        metadata = {
            "filename": image_filename,
            "species_code": filename_species_code,
            "scientific_name": sciname,
            "common_name": comname,
            "caption": caption,
            "image_url": full_image_url,
        }
        #print(metadata)  # You can save this metadata to a file if needed
        """

C:\Users\Walter\AppData\Local\Temp\ipykernel_10152\2203162865.py:7: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\Cleaned_Clements-v2024-October-2024-rev.csv")  # Ensure your DataFrame has 'category', 'species_code', and 'scientific name' columns


        Downloaded: Spotted Nightjar
        Downloaded: Spotted Nightjar
        Downloaded: Solomons Nightjar
        Downloaded: New Caledonian Nightjar
        Downloaded: White-throated Nightjar
        Downloaded: White-throated Nightjar
        Downloaded: Diabolical Nightjar
        Downloaded: Diabolical Nightjar
        Downloaded: Papuan Nightjar
        Downloaded: Papuan Nightjar
        Downloaded: Archbold's Nightjar
        Downloaded: Archbold's Nightjar
        Downloaded: Malaysian Eared-Nightjar
        Downloaded: Malaysian Eared-Nightjar
        Downloaded: Great Eared-Nightjar (cerviniceps)
        Downloaded: Great Eared-Nightjar (cerviniceps)
        Downloaded: Great Eared-Nightjar (Philippine)
        Downloaded: Great Eared-Nightjar (Philippine)
        Downloaded: Collared Nightjar
        Downloaded: Collared Nightjar
        Downloaded: Nacunda Nighthawk
        Downloaded: Nacunda Nighthawk
        Downloaded: Nacunda Nighthawk
        Downloaded: Nacund

KeyboardInterrupt: 